In [2]:
import pandas as pd
import numpy as np
from numpy import linalg
import matplotlib.pyplot as plt
import seaborn as sbn
from scipy.stats import invgamma
import logging

In [3]:
from notebookutils import root_dir; root_dir()

now in dir:  /Users/Jeppe/Projects/BayesFactorModel


In [4]:
from model.utils import read_clean_kv17, read_testdata1, read_party_keys, matrix, vector, party_name_from_key, generate_Tau_trace_df
from model.distributionplotter import DistributionPlotter
from model.traceplotter import TracePlotter
from model.parameterframe import ParameterFrame
from model.parameters import Parameters

In [5]:
np.random.seed(100)

In [6]:
data = read_clean_kv17(drop_party_key = True)
labels = read_party_keys()

In [7]:
data = matrix(data) #making testing different data set simple

(1215, 15)


In [8]:
sigma_array = np.array([np.random.normal()**2 for _ in range(15)])
F = np.matrix([np.random.normal(size=3) for _ in range(1215)])
Beta = np.matrix(np.random.normal(size=(15,3)))
Ystar = np.matrix(np.random.normal(size=(1215,15)))

In [9]:
tau = [- np.inf] + [-2, -0.5, 0.5, 2] + [np.inf]
Tau = dict()
for i in range(15):
    Tau[i] = tau

In [10]:
import numpy as np
from numpy import linalg
from scipy.stats import invgamma
from scipy.stats import truncnorm
import logging

from model.utils import vector


class OrderedProbitGibbsSampler():

    logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

    def __init__(self, n_factors, data, n_answers):

        self.y = data
        self.k = n_factors
        self.p = data.shape[1]
        self.T = data.shape[0]
        self.I_k = np.identity(self.k)
        self.I_p = np.identity(self.p)
        self.n_answers = n_answers
        
        self.v = 0.5
        self.s_sq = 2
        self.C0 = 2
        self.mu0 = 1

        self.Beta_list = list()
        self.Sigma_list = list()
        self.F_list = list()
        self.Tau_list = list()
        self.Ystar_list = list()
        
        print('number of variables:', self.p, ' number of observations:', self.T)

    def f_t(self, Beta, Sigma, t):
        """Posterior of f_t"""

        y_t = self.Ystar[[t]].T
        S_inv = linalg.inv(Sigma * self.I_p)

        scale = linalg.inv(self.I_k + np.dot(np.dot(Beta.T, S_inv), Beta))
        loc = vector(np.dot(np.dot(np.dot(scale, Beta.T), S_inv), y_t))

        return np.random.multivariate_normal(loc, scale)

    def Beta_i(self, Sigma, F, i):

        if i < self.k:

            C_i = self.C_i(F, Sigma, i)
            m_i = self.m_i(C_i, F, Sigma, i)

            B_i = np.random.multivariate_normal(m_i, C_i)
            if B_i[i] < 0:
                # print(B_i[i])  # possible bug
                #B_i = np.random.multivariate_normal(m_i, C_i)
                B_i[i] = 0

            if i < self.k:
                B_i = np.append(B_i, np.zeros(self.k - i - 1))

        elif i >= self.k:

            C_k = self.C_k(F, Sigma, i)
            m_k = self.m_k(C_k, F, Sigma, i)

            B_i = np.random.multivariate_normal(m_k, C_k)

        else:
            raise ValueError('k is {0}, i is {1} - Beta_i probs'.format(self.k, i))

        return vector(B_i)


    def C_i(self, F, Sigma, i):
        """If i <= k """

        F_i = F.T[:i + 1].T
        sigma_i = Sigma[i]
        identity_i = np.identity(i + 1)

        return linalg.inv((1 / self.C0) * identity_i + (1 / sigma_i) * np.dot(F_i.T, F_i))

    def C_k(self, F, Sigma, i):
        """if i > k"""

        sigma_i = Sigma[i]
        return linalg.inv((1 / self.C0) * self.I_k + (1 / sigma_i) * np.dot(F.T, F))

    def m_i(self, C_i, F, Sigma, i):
        """If i <= k """

        F_i = F[:, :i + 1]  # 2000 X i
        sigma_i = Sigma[i]  # 1 x 1
        ones_i = np.matrix(np.ones(i + 1)).T
        y_i = self.Ystar[:, [i]] #changed to Ystar
        tmp = (1 / self.C0) * self.mu0 * ones_i + (1 / sigma_i) * np.dot(F_i.T, y_i)
        return vector(np.dot(C_i, tmp))

    def m_k(self, C_k, F, Sigma, i):
        """if i > k"""

        sigma_i = Sigma[i]  # 1 x 1
        ones_k = np.matrix(np.ones(self.k)).T
        y_i = self.Ystar[:, [i]] #changed to Ystar

        tmp = (1 / self.C0) * self.mu0 * ones_k + (1 / sigma_i) * np.dot(F.T, y_i)
        return vector(np.dot(C_k, tmp))

    def calc_Beta(self):

        B = np.matrix([self.Beta_i(self.Sigma, self.F, i) for i in range(self.p)])
        self.Beta_list.append(B)
        return B

    def calc_F(self):

        F = np.matrix([self.f_t(self.Beta, self.Sigma, t) for t in range(self.T)])
        self.add('F', F)
        return F

    def calc_Ystar(self):
        
        #defined here to not make multiple call of dot product (specially for Psi)
        Y = self.y
        Psi = self.Psi
        
        Ystar = np.matrix([self.calc_Ystar_i(Psi[i], Y[i]) for i in range(self.T)])
        self.add('Ystar', Ystar)
        return Ystar
    
    def calc_Ystar_i(self, Psi_i, Y_i):
            
        Y_star_i = list()
        for question in range(self.p):
            psi_i_j = Psi_i[0,question]
            y_i_j = self.get_y_i_j(Y_i,question)
            lb, ub = self.get_bounds(y_i_j, question)
            Y_star_i.append(self.calc_Ystar_i_j(lb, ub, psi_i_j)) 
            
        return Y_star_i

    def calc_Ystar_i_j(self, lower_bound, upper_bound, mean):
        #weird implementation of python required following:
        ub = upper_bound - mean
        lb = lower_bound - mean
        return truncnorm.rvs(a = lb, b = ub, loc = mean, scale = 1)

    def get_bounds(self, y_i_j, question):
        
        tau = self.Tau[question]
        upper_bound = tau[y_i_j]
        lower_bound = tau[y_i_j - 1]
        return lower_bound, upper_bound
    
    @staticmethod
    def get_y_i_j(Y_i, question):
        """To handle unfortunate 0'es"""
        res = Y_i[0, question]
        if res == 0:
            res = 3
            
        return res
    
    def calc_Tau(self):
        
        Tau = dict()
        for question in range(self.p):
            Tau[question] = self.calc_Tau_p(question)
            
        self.add('Tau', Tau)
        return Tau
    
    def calc_Tau_p(self, p):
        """Cutpoints/answers for given quiestion p"""
        
        _tau = [self.calc_tau_l_p(p, l) for l in range(1, self.n_answers)]
        return [- np.inf] + _tau + [np.inf]
    
    def calc_tau_l_p(self, p, l):
        
        _ , Ystar_max = self.get_Ystar_min_max(p, l) 
        Ystar_min, _ = self.get_Ystar_min_max(p, l + 1)
        l_plus = self.get_tau_l_p(p, l+1)
        l_minus = self.get_tau_l_p(p, l-1)
        _min = min(Ystar_min, l_plus)
        _max = max(Ystar_max, l_minus)
        
        #print('tau', 'p=',p,', l=',l)
        #print(l_plus, Ystar_min)
        #print(l_minus, Ystar_max)
        
        try:
            return np.random.uniform(_min, _max)
        except OverflowError as e:
            print(_min, _max)
            raise e
        
    def get_Ystar_min_max(self, question, answer):
        """
        answer is the answer of question (1, 2, 3, 4, 5)"""
        Y_l = self.y[:,question]
        Ystar_l = self.Ystar[:,question]
        
        #Ystar observation where y_l == 0
        Ystar_legal = []
 
        for i in range(self.T):
            if Y_l[i] == answer :
                Ystar_legal.append(Ystar_l[i])
        return float(min(Ystar_legal)), float(max(Ystar_legal))
    
    def get_tau_l_p(self, p, l):
        return self.Tau[p][l]
        
    @property
    def Beta(self):
        return self.Beta_list[-1]

    @property
    def F(self):
        return self.F_list[-1]

    @property
    def Sigma(self):
        #return self.Sigma_list[-1]
        return [1 for _ in range(self.p)]

    @property
    def Psi(self):
        return np.dot(self.F ,self.Beta.T)
    
    @property
    def Tau(self):
        return self.Tau_list[-1]

    @property
    def Ystar(self):
        return self.Ystar_list[-1]
    
    def add(self, param, value):
        """ add to Sigma_list, Beta_list or F_list

        Parameters
        ==========
        param: (str)
            string that should be of {'Sigma', 'F', 'Beta'}
        value: (obj)
            appropriate object for given list

        """

        if param == 'Sigma':
            self.Sigma_list.append(value)
            
        elif param == 'Tau':
            self.Tau_list.append(value)
            
        elif param == 'Ystar':
            self.Ystar_list.append(value)

        elif param == 'F':
            self.F_list.append(value)

        elif param == 'Beta':
            self.Beta_list.append(value)

        else:
            raise ValueError("Param must be in {'F', 'Sigma', 'Beta', 'Ystar', 'Tau'}")

    def sampler(self, n_iterations):

        logging.info("Sampling begins")
        for i in range(n_iterations):

            self.calc_Ystar()
            self.calc_Tau()
            self.calc_F()
            self.calc_Beta()
            if (i % 5 == 0):
                logging.info("run {0} simulations".format(i))


In [11]:
gs = OrderedProbitGibbsSampler(3, data, n_answers = 5)
gs.add('Sigma',sigma_array)
gs.add('F', F)
gs.add('Beta',Beta)
gs.add('Tau', Tau)

number of variables: 15  number of observations: 1215


In [12]:
gs.sampler(3)

2018-11-18 15:45:58,553 : INFO : Sampling begins
2018-11-18 15:46:02,219 : INFO : run 0 simulations


In [13]:
beta_params = ParameterFrame(gs.Beta_list, 'beta')
beta_trace_estimation = beta_params.get_trace_df()
beta_trace_estimation.to_pickle('data//probit_v2_estimation_beta_trace_df.pkl')

In [14]:
factor = ParameterFrame(gs.F_list, 'factor_estimation')
factor_trace_estimation = factor.get_trace_df()
factor_trace_estimation.to_pickle('data//probit_v2_estimation_factor_trace_df.pkl')

In [15]:
ystar = ParameterFrame(gs.Ystar_list, 'ystar')
ystar_trace_estimation = ystar.get_trace_df()
ystar_trace_estimation.to_pickle('data//probit_v2_ystar_trace_df.pkl')

In [16]:
tau_trace = generate_Tau_trace_df(gs.Tau_list)
tau_trace.to_pickle('data//probit_v2_tau_trace_df.pkl')